In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
file_path = 'Final Year Project/dataset.csv'
data = pd.read_csv(r"C:/Users/jorge/Project_1/Final Year Project/dataset.csv")

# Display the first few rows of the dataset
data.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [3]:
data = pd.read_csv(r"C:/Users/jorge/Project_1/Final Year Project/dataset.csv")

# If your text column is named "tweet":
training_sentences = data["tweet"].astype(str).tolist()

# 3) Create & Fit the Tokenizer
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)


# 4) Convert Text to Sequences & Pad
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')


In [4]:
# Load GloVe embeddings
embedding_index = {}
embedding_dim = 100  # Set this to match the dimensionality of your GloVe embeddings (e.g., 50, 100, 200, 300)
with open(r'C:\Users\jorge\Desktop\Python\glove.6B\glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:  # Indentation should start here
        values = line.split()
        word = values[0]
        coefs = np.array(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [5]:
# Vocabulary size and embedding dimensions
vocab_size = 10000  # or the vocab size used in your tokenizer

# Initialize the embedding matrix with zeros
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Fill the embedding matrix with GloVe vectors for words in our vocabulary
for word, index in tokenizer.word_index.items():
    if index < vocab_size:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [6]:
# Extract tweets and labels
tweets = data['tweet'].astype(str).values
labels = data['class'].values

# Split data into training and validation sets
tweets_train, tweets_val, labels_train, labels_val = train_test_split(tweets, labels, test_size=0.2, random_state=42)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(tweets_train)

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(tweets_train)
val_sequences = tokenizer.texts_to_sequences(tweets_val)


In [7]:
# Build a Sequential model
# Build a Sequential model with regularization
# Build a Sequential model with the pre-trained embedding layer
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, 
                              output_dim=embedding_dim, 
                              weights=[embedding_matrix],  
                              trainable=False),  # Set to True if you want to fine-tune
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, kernel_regularizer=regularizers.l2(0.001))),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [8]:
X_train, X_val, y_train, y_val = train_test_split(
    padded_sequences,
    labels,
    test_size=0.2,
    random_state=42
)
# Train the model
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Train the model with early stopping
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Epoch 1/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8159 - loss: 0.7005 - val_accuracy: 0.8777 - val_loss: 0.3952
Epoch 2/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.8783 - loss: 0.3936 - val_accuracy: 0.8818 - val_loss: 0.3553
Epoch 3/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.8854 - loss: 0.3453 - val_accuracy: 0.8872 - val_loss: 0.3389
Epoch 4/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.8896 - loss: 0.3266 - val_accuracy: 0.8907 - val_loss: 0.3271
Epoch 5/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.8910 - loss: 0.3281 - val_accuracy: 0.8892 - val_loss: 0.3274
Epoch 6/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.8988 - loss: 0.3065 - val_accuracy: 0.8915 - val_loss: 0.3248
Epoch 7/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.8951 - loss: 0.3131 - val_accuracy: 0.8949 - val_loss: 0.3203
Epoch 8/10
620/620 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9008 - loss: 0.3024 - 

In [9]:
# Evaluate the model
loss, accuracy = model.evaluate(X_train,y_train,)
print(f"Validation Accuracy: {accuracy}")

620/620 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9008 - loss: 0.2807
Validation Accuracy: 0.9031574726104736


In [10]:
model.save('offensive_language_detection_model.keras')

In [11]:
import tensorflow as tf

model = tf.keras.models.load_model("offensive_language_detection_model.keras")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 100)            │       1,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 128)                 │          84,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,278,795 (4.88 MB)

 Trainable params: 92,931 (363.01 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

 Optimizer params: 185,864 (726.04 KB)

In [14]:
# 1. Load the tokenizer
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

# 2. Load the model
model = tf.keras.models.load_model("offensive_language_detection_model.keras")

# 3. Load foul words from en.txt
with open("en.txt", "r", encoding="utf-8") as f:
    foul_words = f.read().splitlines()

# 4. Dictionary check function
def contains_offensive_word(text, foul_list):
    words = text.lower().split()
    for w in words:
        w = w.strip('.,!?;:"\'')
        if w in foul_list:
            return True
    return False

# 5. Combined classification function with probability printing
def classify_text(text, tokenizer, model, foul_list, max_length=100, threshold=0.5):
    # A) Dictionary-based check
    if contains_offensive_word(text, foul_list):
        print(f"Probability: (dictionary override) -> 1.0")
        return "offensive"

    # B) Model-based prediction
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=max_length, padding='post', truncating='post')
    prob = model.predict(padded_seq)[0][0]

    # Print the probability before deciding the label
    print(f"Probability: {prob:.4f}")

    # Apply threshold
    return "offensive" if prob > threshold else "not offensive"

# 6. Test with some sample sentences
test_sentences = [
    "hello friend, how fking dumb are you ",
    "you are so bitch",
    "you cute ass",
    "shut the fuck up"
]

for s in test_sentences:
    print(f"Text: {s}")
    label = classify_text(s, tokenizer, model, foul_words, max_length=100, threshold=0.2)
    print(f"Label: {label}\n")


Text: hello friend, how fking dumb are you 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
Probability: 0.1649
Label: not offensive

Text: you are so bitch
Probability: (dictionary override) -> 1.0
Label: offensive

Text: you cute ass
Probability: (dictionary override) -> 1.0
Label: offensive

Text: shut the fuck up
Probability: (dictionary override) -> 1.0
Label: offensive

